## Piranha 

This is a guide for developing automation for ***cleaning up code related to stale feature flags*** 

Languages supported : Java, Kotlin, Go and Swift


ℹ️ **_NOTE:_** This tutorial is in Python for pedagogical purposes. 
Piranha can be used with the CLI or using the Rust API. 
We would encourage following this tutorial even if you do not intend to use the Python API.

Now let's set up the notebook by installing the polyglot-piranha from source. 

In [ ]:
# Please install [rust](https://rustup.rs/)
! python3 -m venv ../.env
! cd ../ && cargo build --no-default-features && maturin develop

from polyglot_piranha import PiranhaArguments, execute_piranha, RuleGraph, Rule, OutgoingEdges

<details>
    <summary><b>CLI</b></summary>
    <p> <b>Build the executable from source</b> </p>
    <code>
    cargo build --no-default-features -r
    </code>
    <p> <i>The executable <code>polyglot_piranha</code> will be available under <code>target/release</code></i> </p>
</details>

Now let's take the below example where we want to clean up the code related to the stale flag `SOME_STALE_FLAG` 🏴 because it has been permanently enabled.

In [ ]:
input_code = """package com.uber.piranha;
class SampleJava {
  
  // An enum that declares the feature flag 
  enum FeatureFlag {
    SOME_STALE_FLAG,
  }

  public void sampleMethod(ExperimentInterface exp) {
    if (exp.isToggleEnabled(SOME_STALE_FLAG)) {
      System.out.println("SOME_STALE_FLAG is enabled");
    } else {
      System.out.println("SOME_STALE_FLAG is disabled");
    }
  }
}
"""

<details>
    <summary><b>CLI</b></summary>
    <p> <b>A similar example for CLI is available <a href="./feature_flag_cleanup/java/SampleClass.ja"> here </a></b> </p>
    <code>
    cargo build --no-default-features -r
    </code>
    <p> <i>The executable <code>polyglot_piranha</code> will be available under <code>target/release</code></i> </p>
</details>


We will now craft a Piranha rule to update the feature flag API `exp.isToggleEnabled(SOME_STALE_FLAG)` and delete the enum case `SOME_STALE_FLAG`

In [ ]:
# a rule to update `exp.isToggleEnabled(SOME_STALE_FLAG)`
update_rule_toggle_enabled = Rule (
    name= "update_toggle_enabled",
    query= """((
    (method_invocation 
        name : (_) @name
        arguments: ((argument_list 
            ([
                (field_access field: (_)@argument)
                (_) @argument
            ])) )
    ) @method_invocation)
    (#eq? @name "isToggleEnabled")
    (#eq? @argument "@stale_flag_name")
    )""",
    replace_node="method_invocation",
    replace="@is_treated",
    holes= set(["stale_flag_name", "is_treated"]),
    # Try commenting out the below line and see the difference in output
    groups= set(["replace_expression_with_boolean_literal"])
)

# a rule to delete enum case `SOME_STALE_FLAG`
delete_enum_case = Rule (
    name = "delete_enum_constant",
    query = """(
        ((enum_constant name : (_) @enum_case) @ec)       
        (#eq? @enum_case  "@stale_flag_name")
    )""",
    replace_node = "ec",
    replace = "",
    holes = set(["stale_flag_name"]),
    # Try commenting out the below line and see the difference in output
    groups = set(["delete_enum_entry"]),
)

rule_graph= RuleGraph(rules=[update_rule_toggle_enabled, delete_enum_case], edges=[])

piranha_args = PiranhaArguments(
        language = "java",
        code_snippet= input_code,
        rule_graph= rule_graph,
        substitutions= {'stale_flag_name': 'SOME_STALE_FLAG', 'is_treated': 'true'},
        dry_run= True,
        cleanup_comments = True
)
output = execute_piranha(piranha_args)
output_content= output[0].content
assert output_content != input_code # sanity test for this tutorial
print(output_content)

The rule `update_rule_toggle_enabled` specifies a rule that matches against an expression like `exp.isTreated(SOME_STALE_FLAG)` and replaces it with `true`. 
 
<details>
  <summary>See the rule</summary>

  ##### `update_rule_toggle_enabled`
  ```python Rule (
    name= "update_toggle_enabled",
    query= """((
    (method_invocation 
        name : (_) @name
        arguments: ((argument_list 
            ([
                (field_access field: (_)@argument)
                (_) @argument
            ])) )
    ) @method_invocation)
    (#eq? @name "isToggleEnabled")
    (#eq? @argument "@stale_flag_name")
    )""",
    replace_node="method_invocation",
    replace="@is_treated",
    holes= set(["stale_flag_name", "is_treated"])
    # Try un-commenting the below line and see the difference in output
    groups= set(["replace_expression_with_boolean_literal"])
)
  ```
</details>


The **`query`** property of the rule is a [tree-sitter query](https://tree-sitter.github.io/tree-sitter/using-parsers#pattern-matching-with-queries) that is matched against the source code to identify specific AST patterns.

ℹ️ **_NOTE:_** Use [tree-sitter playground](https://tree-sitter.github.io/tree-sitter/playground) to craft the query that matches your specific APIs. This playground allows a user to create a query and match it against some input source code. This playground "syntax highlights" in the input code snippet based on input query. 
<details>
<summary>Using the tree-sitter play ground locally </summary>
Install tree-sitter [CLI](https://github.com/tree-sitter/tree-sitter/blob/master/cli/README.md).
```bash
git clone https://github.com/tree-sitter/tree-sitter-java.git # Or any other tree-sitter language repo
cd tree-sitter-java
tree-sitter generate 
tree-sitter build-wasm
tree-sitter playground
```
</details>

The node captured by the tag-name specified in the **`replace_node`** property is replaced with the pattern specified in the **`replace`** property.
The **`replace`** pattern can use the tags from the **`query`** to construct a replacement based on the match (somewhat similar to [regex-replace](https://docs.microsoft.com/en-us/visualstudio/ide/using-regular-expressions-in-visual-studio?view=vs-2022)).

Each rule also contains the **`groups`** property, that specifies the kind of change performed by this rule. Based on this group, appropriate
cleanup will be performed by Piranha. For instance, `replace_expression_with_boolean_literal` will trigger deep cleanups to eliminate dead code (like eliminating `consequent` of a `if statement`) caused by replacing an expression with a boolean literal.
Currently, Piranha provides deep clean-ups for edits that belong the groups - `replace_expression_with_boolean_literal`, `delete_enum_entry`, and `delete_method`. Adding an appropriate entry to the groups, hooks up the user defined rules to the pre-built cleanup rules.

The property **`holes`** specify the holes or template variables `@treated` and `@stale_flag_name` that need to be replaced with some concrete values so that the rule matches only the feature flag API usages corresponding to a specific flag, and replace it specifically with `true` or `false`.

The values for these template variables is specified via the `substitutions` dictionary. This dictionary is required to construct piranha arguments. 

<details>
    <summary><b>CLI</b></summary>
    <h4>Java</h4>
    <code>
    ./target/release/polyglot_piranha -c demo/feature_flag_cleanup/java -f demo/feature_flag_cleanup/java/configurations -l "java" -s stale_flag_name=SAMPLE_STALE_FLAG -s treated=true -s treated_complement=false
    </code>
    
    <h4>Kotlin</h4>
    <code>
    ./target/release/polyglot_piranha -c demo/feature_flag_cleanup/kt -f demo/feature_flag_cleanup/kt/configurations -l "kt" -s stale_flag_name=SAMPLE_STALE_FLAG -s treated=true -s treated_complement=false
    </code>
</details>